# Mathematics of Poker

### Part 1, Chapter 2:  Predicting the Future and Sample Outcomes

-----------------------

#### Calculating Winning Sessions from Player Stats
Page 28

In [33]:

from scipy.stats import norm # for norm.cdf to calculate the cumulative normal distribution function
from numpy import std, sqrt # standard deviations, square root functions for probability calculations

def win_session_probability(winrate, standard_deviation, hands_played, greater_than=0):
    """Calculate the probability that a given poker session will be a winning one given some player statitics
    
    Parameters
    ----------
    
    winrate: float
        the player winrate in units per hand. This is also the expectation. 
        
    standard_deviation: float 
        the standardard deviation over the same interval of hands used to calculate the winrate
        for example, if you have play 10 hands and have winnings of [0, 1, 1, 0, -2, 10, -2, 0, 0, 1] = 9 units
        then your winrate = 9 / 10 => 0.9 units per hand and your standard deviation is 3.38
        
    hands_played: int
        the number of hands for which we will provide an estimate
        
    greater_than: int
        default to 0
        The outcome we are interested in
        if set to x, this function returns the probability that the player earns greater than x units
        
        
    Returns
    -------
    
    probability: float
        the probability the player has a winning session. 
        This is equal to:
            1 - phi(z_score)    
    """
    
    # first we need to get the expectation over the sample 
    # this is just equal to the winrate * sample_size
    
    expected_value_over_sample = winrate * hands_played

    
    
    # next we will calculate the standard deviation over hands_played
    
    sigma_over_hands = sqrt(hands_played) * standard_deviation
    
    
    
    # now that we have both of these values, we can calculate the z-score
    # the z-score is passed into the CDF to get the probability estimate
    
    z_score = (greater_than - expected_value_over_sample) / sigma_over_hands # 0 here because we care only about winning or losing
    
    
    # pass that to the CDF function
    
    phi_of_z = norm.cdf(z_score)
    
    
    # we take 1 - phi_of_z because that function returns the probability that the outcome will be LESS than 0
    return 1-phi_of_z



# verify against the page in the book
win_session_probability(0.015, 2, 300)

# 0.5517 is correct

0.5516787352796411

Now we can answer some questions. 

**What is the probability a player with these statistics wins 100 units over 3000 hands?**

In [3]:
win_session_probability(winrate=0.015, standard_deviation=2, hands_played=3000, greater_than=100)

0.30780597184951286

This player should expect to win more than 100 units over 3000 hands about 30% of the time with a winrate of 0.015 units per hands and a standard deviation of 2 over the same interval. 

#### Heads Up All In: The AK v AQ 50/50 Story
---------------

One player claimed that over a sample of around 2000 hands, AK v AQ were running 50-50. Is this player telling the truth? is the site rigged? we don't know, but how likely is this? 

In [30]:
# AK v AQ is about 73.5% all in preflop. 
# If we assign a value of 1 when AK wins and 0 when AQ wins, then the expectation (or mean) is 0.735
# we'll get the probability that AQ wins 50% or more of the time


def check_ak_aq():
    """Proceed as we did in the equation above"""
    
    NUM_HANDS = 2000
    
    
    # first we get the mean
    expectation = 0.735
    expectation_over_sample = expectation * NUM_HANDS # since this scenario has 2000 hands 
    
    # next we need to calculate the standard deviation over this sample 
    # Variance = prob_win*(1-expectation)^2 + prob_lose(0-expectation)^2 => sigma^2
    
    variance = 0.735 * (1-expectation)**2 + (1 - 0.735) * (0 - expectation )**2
    sigma = sqrt(variance)
    
    sigma_over_hands = sqrt(NUM_HANDS) * sigma
                        
        
    # then we get the z score
    # we use NUM_HANDS*.5 = 1000 here because we are interested in the chance that AQ wins more than 50% or 1000 hands
    
    z_score = (1000 - expectation_over_sample) / sigma_over_hands
    print("Number of Standard Deviations from the mean estimate:", abs(z_score), "\n")
    
    # then we calculate phi(z_score)
    
    print("Probability: ")                    
    return norm.cdf(z_score)
    

In [32]:
# check the results
check_ak_aq()

Number of Standard Deviations from the mean estimate: 23.81311742871408 

Probability: 


1.221311050119768e-125